In [ ]:
import numpy as np
import sys
import tensorflow as tf
tf.enable_eager_execution()
import tensorflow.keras as keras

### Tic-Tac-Toe value approximation

In [ ]:
CODE_MSB_VALUE = 1<<3
def codeBoard(board):
    return np.array(list(map(
        lambda L:list(map(lambda l: [int(x) for x in bin((1<<l)+CODE_MSB_VALUE)[3:]], L)),
        board
    )))

def getRot3(board):
    rotations = [board]
    for i in range(3):
        rotations.append(np.rot90(rotations[-1]))
    return rotations

def augment(board):
    flip = np.transpose(board)
    return [codeBoard(v) for v in getRot3(board)+getRot3(flip)]

In [ ]:
board = [[0,0,1,2],
         [1,1,2,0],
         [0,0,2,1],
         [1,2,0,0]
        ]

code = np.array(augment(board), dtype=np.float32)
segment_hight = code.shape[1]
segment_width = code.shape[2]
code_depth = code.shape[3]

#### actions iterator

In [ ]:
def getOneMoveTransition(board, move):
    player, location = move
    is_terminal = isStraitConnection(board, location, player)
    next_board = board.copy()
    next_board[location] = player
    return (is_terminal, next_board)

def applyMove(board, move):
    location, player = move
    next_board = board.copy()
    next_board[location] = player
    return (is_terminal, next_board)


#### test the environment

In [ ]:
[(getOneMoveTransition(np.array(board),(2,loc))[0], loc) for loc in getVacationsIterator(board)]

#### build the model

In [ ]:
class vModel(tf.keras.Model):
    def __init__(self):
        super(vModel, self).__init__()
        self.dense1 = tf.keras.layers.Dense(units=44, name='dense1')
        self.dense2 = tf.keras.layers.Dense(units=1, name='dense2')
    
    def call(self, input):
        x = self.dense1(input)
        x = self.dense2(x)
        return tf.math.reduce_max(x, axis=0)

model = vModel()

#### get the gradient

In [ ]:
def grad(position, estimate):
    with tf.GradientTape() as t:
        y = model(position)
        loss = tf.square(y, estimate)
        grad = t.gradient(loss, model.trainable_variables)

<font color=#404040>
Some testing stuff
</font>

In [ ]:
l = [33,6,-22,0]
l.remove(0)
l

<font color=#404040>
Some obsolete stuff
</font>

In [ ]:
k2D = np.array([[[2, 1, 0],[6, 0, 1],[2, 1, 0],[0,-1, 1]],
               [[2, 1, 3],[9,-7, 1],[2, 1,-9],[5, 0, 0]],
               [[0, 1, 3],[0, 0, 1],[2, 1,-3],[5,-1, 1]],
               [[2, 1,-3],[0,-2, 1],[2, 1, 3],[4, 0,11]]
              ], dtype=np.float32)
kernel2D = tf.reshape(k2D, k2D.shape+(1,), name='kernel2D')

#reshape for 2D convolution
code2D = code.reshape((1,code.shape[0]*code.shape[1],)+code.shape[2:])
print(tf.squeeze(tf.nn.conv2d(code2D, filters=kernel, strides=4, padding='VALID')).numpy())

In [ ]:
k1D = np.array([[2, 1, 3],[9, 0, 1],
                [0,-1, 4],[5, 0,-3]], dtype=np.float32)
kernel1D = tf.reshape(k1D, k1D.shape+(1,), name='kernel1D')
#reshape for 1D convolution
code1D = code.reshape((1,code.shape[0]*code.shape[1]*code.shape[2],code.shape[3]))
print(tf.squeeze(tf.nn.conv1d(code1D, filters=kernel1D, stride=4, padding='VALID')).numpy())

In [ ]:
Proof Number Search implementation
def descendToMPN(node):
    if not node.isExpanded():
        node.expand()
        node.update()
        return True
    
    select = []
    if node.isAnd():
        dn = INF
        for child in node.children:
            if child.disproof_num < dn:
                dn = child.disproof_num
                select = [child]
            # no need anymore:
            #elif child.disproof_num == dn: 
            #    select.append(child)
    else:
        pn = INF
        for child in node.children:
            if child.proof_num < pn:
                pn = child.proof_num
                select = [child]
            # no need anymore:
            #elif child.proof_num == pn:
            #    select.append(child)
    expanded = False
    for child in select:
        expanded = descendToMPN(child)
        if expanded: break

    node.update()
    return expanded

def iteratePNSearch(root, max_iterations=None, max_nodes=None):
    n=0
    while (not max_nodes or root.count < max_nodes) and \
        (not max_iterations or n < max_iterations):
        if not descendToMPN(root):
            print("\nNo expansion. Search terminated.")
            break
        if root.proof_num == 0:
            print("\nprooved")
            break
        elif root.disproof_num == 0:
            print("\ndisprooved")
            break
        else:
            n += 1
            print("\rIteration {:4}: nodes count is {:5}".format(n,root.count), 
                  flush=True, end='')
    print("\nIteration {:4}: nodes count is {:5}".format(n,root.count))
    
INF2 = sys.maxsize